In [4]:
import zstandard as zstd
import json
import io

In [2]:
# extract smples

In [11]:
def extract_samples(file_path, num_samples=20):
    samples = []
    with open(file_path, 'rb') as fh:
        dctx = zstd.ZstdDecompressor()
        # Create a streaming reader
        stream_reader = dctx.stream_reader(fh)
        # Wrap in a TextIOWrapper to read lines incrementally
        text_stream = io.TextIOWrapper(stream_reader, encoding='utf-8')
        
        for i, line in enumerate(text_stream):
            if i >= num_samples:
                break
            if line.strip():  # Skip empty lines
                try:
                    samples.append(json.loads(line))
                except json.JSONDecodeError:
                    print(f"Error decoding JSON on line {i}")
    return samples

# Use raw strings for Windows paths
submission_file = r"C:\Users\Administrator\Desktop\RS_2019-04.zst" # submission data
comment_file = r"C:\Users\Administrator\Desktop\RC_2019-04.zst" # comment data

# Extract samples (20k)
submission_samples = extract_samples(submission_file, 20000)
comment_samples = extract_samples(comment_file, 20000)

# Save to files
save_submission_samples = r"C:\Users\Administrator\Desktop\deep_alignment\submission_samples_20k.json"
save_comment_samples = r"C:\Users\Administrator\Desktop\deep_alignment\comment_samples_20k.json"
with open(save_submission_samples, "w") as f:
    json.dump(submission_samples, f, indent=2)

with open(save_comment_samples, "w") as f:
    json.dump(comment_samples, f, indent=2)

# print("Success! Samples saved to JSON files 20k Yeah.")

Success! Samples saved to JSON files 20k Yeah.


In [1]:
# convert extracted comments to the unified schema

In [12]:
def safe_get(data, key, default=None):
    value = data.get(key, default)
    return value if value is not None else default
    
def transform_comment(comment):
    return {
        "user_id": "usr_" + safe_get(comment, "author_fullname", "empty_body"),
        "chunk_id": safe_get(comment, "permalink", "conv_unknown"),
        "platform": "reddit",
        "content": [
            {
                "speaker": "SELF",
                "text": safe_get(comment, "body", "empty_content")
            }
        ],
        "timestamp": safe_get(comment, "created_utc", "unknown_timestamp")
    }

# Load the saved comment samples
with open(save_comment_samples, "r") as f:
    comment_samples = json.load(f)

# Transform all comments
transformed_comments = [transform_comment(comment) for comment in comment_samples]

# Save transformed data
save_transformed_samples = r"C:\Users\Administrator\Desktop\deep_alignment\transformed_comments_20k.json"
with open(save_transformed_samples, "w") as f:
    json.dump(transformed_comments, f, indent=2)

# print(f"Transformed {len(transformed_comments)} comments successfully")

Transformed 1000 comments successfully 20k !
